In [ ]:
import pandas as pd
import numpy as np

import os
from kaggle.api.kaggle_api_extended import KaggleApi
import time

In [ ]:
version = 'vxx' # UPDATE

In [ ]:
base_path = '/home/clas_giulia_s/buckets/b1/'

files_path = base_path + f'exp/competencia_2/{version}/'

competition = 'dm-ey-f-2024-segunda'


In [ ]:
puntos_corte = np.linspace(9000, 13000, 500).astype(int)
puntos_corte

In [ ]:
# diccionario con la semilla y el corte de cada version
combinations = {}

counter = 1
for corte in puntos_corte:
    combinations[counter] = {'seed':'semillero 100','corte':int(corte)}
    counter += 1

# path de cada version        
files = [f'{files_path}results_{version}_{i}.csv' for i in range(1,51)]

# descripcion de cada edicion
descriptions= [
    f"seed = {combinations[i]['seed']}, corte = {combinations[i]['corte']}"
    for i in range(1, len(combinations)+1)
]

In [ ]:
api = KaggleApi()
api.authenticate()

In [ ]:
archivos = []
scores = []
descripciones = []

for i in range(len(files)): # UPDATE!

    submitted = False
    while submitted == False:
        try:
            api.competition_submit(file_name=files[i],
                                   message=descriptions[i],
                                   competition=competition)
        except Exception as e:
            print(f'Error al subir {files[i]}: {e}')
            time.sleep(1000)
        else:
            submitted = True 

            # Esperamos un poco para que kaggle no tire error
            time.sleep(60)

            submissions = api.competition_submissions(competition, page_size=1) # traigo solo la ultima
            ultima = submissions[0]
            
            archivo = ultima.fileName
            public_score = ultima.publicScore
            descripcion = ultima.description
            
            archivos.append(archivo)
            scores.append(public_score)
            descripciones.append(descripcion)
            
            print(f'{archivo} subido con exito, public score:{public_score}')

In [ ]:
resultados = pd.DataFrame({
    'Archivo': archivos,
    'Public score': scores,
    'Description':descripciones
})

df = pd.DataFrame({
    'seed': np.repeat('semillero', len(puntos_corte)),
    'corte': np.tile(puntos_corte, 1)
})

resultados = pd.concat([resultados, df], axis=1)

In [ ]:
resultados.to_excel(f'{files_path}scores_{version}.xlsx', index=False)